# Sign language project using Tensorflow 

## Goal

This notebook is to build a sign language classifier. One will be able to create own data, label it and train the classifier on specific signs. We will work with the American Sign Language (ASL) codes.


**Import libraries**

In [1]:
import uuid
import numpy as np
import os
import time
from pathlib import Path
import cv2 
import random
import shutil
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

**Note:** images should have been collected.


### Label collected data

In [2]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']

In [3]:
for label in labels:
    i = 0
    label_path = os.path.join('workspace1', 'images', 'collectedimages', label)

    for filename in os.listdir(label_path):
        my_dest = label + str(i) + ".jpg"
        my_source = os.path.join(label_path, filename)
        my_dest = os.path.join(label_path, my_dest)
        # rename() function will rename all the files
        os.rename(my_source, my_dest)
        i += 1 

**Split data to train/test folder**

In [4]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']
source_path = os.path.join('workspace1', 'images')
        
paths = ['train', 'test']
for p in paths:
    for label in labels:
        path = os.path.join(source_path, p, label)
        if not os.path.exists(path):
            !mkdir {path}  

In [5]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']
number_imgs = 10
test_size = int(np.ceil(0.25*number_imgs))
train_size = number_imgs - test_size

for label in labels:
    train_path = os.path.join('workspace1', 'images', 'train', label)
    test_path = os.path.join( 'workspace1', 'images', 'test', label)

    label_path = os.path.join('workspace1', 'images', 'collectedimages', label)
 
    file_list = os.listdir(label_path)
    
    for file_name in random.sample(file_list, train_size):
        shutil.copy(os.path.join(label_path, file_name), train_path)
    
    for file_name in random.sample(file_list, test_size):
        shutil.copy(os.path.join(label_path, file_name), test_path)


In [6]:
train_path = os.path.join('workspace1', 'images', 'train')
test_path = os.path.join( 'workspace1', 'images', 'test')


In [7]:
train_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

imgs, labels = next(train_batches)

Found 50 images belonging to 5 classes.
Found 36 images belonging to 5 classes.
